In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from functools import reduce
from django.core.serializers import serialize

In [2]:
from tra.models import *
from tra.test_runs_processing import *
from rep_portal.api import *

In [3]:
from tra.storage import get_storage_instance
from tra import utils

In [5]:
def sync_suspension_status_of_test_instances_by_testset_filter(testset_filter_id: int, limit=None):
    def get_ti_suspended_status_from_results(ti_id: int):
        return next((item["suspended"] for item in results if item["id"] == ti_id), None)

    not_found = []
    testset_filter = TestSetFilter.objects.get(id=testset_filter_id)
    postfix = testset_filter.test_lab_path.split('\\')[-1]
    if not limit:
        limit = testset_filter.limit
    results = RepPortal().get_data_from_testinstances(limit=limit, test_lab_path=testset_filter.test_lab_path)
    print(testset_filter.test_lab_path)
    display(results)
    for test_instance in testset_filter.test_instances.all():
        suspended_status = get_ti_suspended_status_from_results(test_instance.rp_id)
        if suspended_status is None:
            print(test_instance.rp_id)
            not_found.append(test_instance.rp_id)
        test_instance.execution_suspended = suspended_status
        test_instance.save()
    return not_found


# def sync_suspension_status_of_test_instances_by_all_testset_filters(query_limit: int=None):
#     testset_filters = TestSetFilter.objects.all()
#     status_response = {}
#     for testset_filter in testset_filters:
#         if testset_filter.is_subscribed_by_anyone(): 
#             status_response[testset_filter.id] = "sync scheduled" 
#             sync_suspension_status_of_test_instances_by_testset_filter(testset_filter_id=testset_filter.id, limit=query_limit)
#         else:
#             status_response[testset_filter.id] = f"TestSetFilter.id: {testset_filter.id} has 0 subscribers - will be skipped" 
#     return status_response

# sync_suspension_status_of_test_instances_by_all_testset_filters()

sync_suspension_status_of_test_instances_by_testset_filter(11)

Starting new HTTPS connection (1): rep-portal.wroclaw.nsn-rdnet.net


Root\Test_Sets\Trunk\RAN_L2_SW_KRK_2\5GC002175_Spillover_from_SA_Mobility_(5GC000704_5GC000708)


[{'id': 10595982,
  'test_set': {'name': '5GC002175-QB'},
  'url': '/qc/instance/details/10595982/',
  'status_color': 'green',
  'last_passed': {'timestamp': '2022-09-08T14:50:32',
   'sw_build': 'SBTS00_ENB_9999_220903_000004'},
  'name': '[1][1.0] 5GC002175-QB-0002_NGAP_UE_Context_Release_Command_protocol_error',
  'status': 'Passed',
  'release': '5G00',
  'test_lvl_area': 'EnTe - Functional Test',
  'sw_build': 'SBTS00_ENB_9999_220903_000004',
  'm_path': 'Root\\Test_Sets\\Trunk\\RAN_L2_SW_KRK_2\\5GC002175_Spillover_from_SA_Mobility_(5GC000704_5GC000708)',
  'suspended': False},
 {'id': 10595983,
  'test_set': {'name': '5GC002175-RB'},
  'url': '/qc/instance/details/10595983/',
  'status_color': 'green',
  'last_passed': {'timestamp': '2022-09-08T15:15:55',
   'sw_build': 'SBTS00_ENB_9999_220903_000004'},
  'name': '[1][1.0] 5GC002175-RB-0002_NGAP_Error_Indication_other_protocol_error',
  'status': 'Passed',
  'release': '5G00',
  'test_lvl_area': 'EnTe - Functional Test',
  'sw_b

[]